In [ ]:
pip freeze

In [1]:
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine
import matplotlib
import pyspark
import pyspark.pandas as ps

ModuleNotFoundError: No module named 'pandas'

In [3]:
pd.set_option('display.max_columns', 100)

In [34]:
engine = create_engine('postgresql+psycopg2://postgres:@localhost/bdtest')

In [36]:
words.to_sql('pandas_db', engine)

644

In [28]:
words_df = pd.read_csv("C:/Users/vovad/Downloads/OnlineNewsPopularity/OnlineNewsPopularity/OnlineNewsPopularity.csv", usecols=[' kw_avg_avg', ' is_weekend', \
' data_channel_is_entertainment', ' data_channel_is_tech', ' data_channel_is_socmed', ' self_reference_avg_sharess', ' LDA_02', ' self_reference_min_shares', \
' kw_max_max', ' n_tokens_content', ' min_positive_polarity', ' kw_min_max', ' num_keywords', ' title_sentiment_polarity', ' kw_max_min', ' shares'])

In [212]:
len(words_df)

300

In [179]:
cols = ['kw_avg_avg', 'is_weekend', 'data_channel_is_entertainment', 'data_channel_is_tech', \
'data_channel_is_socmed', 'self_reference_avg_sharess', 'LDA_02', 'self_reference_min_shares', \
'kw_max_max', 'n_tokens_content', 'min_positive_polarity', 'kw_min_max', 'num_keywords', 'title_sentiment_polarity', 'kw_max_min']

In [29]:
words_df.loc[words_df[" shares"] >= 1400, "Label"] = 1
words_df.loc[words_df[" shares"] < 1400, "Label"] = 0

In [30]:
words = words_df.drop(' shares', axis=1)

In [31]:
words.head()

,n_tokens_content,num_keywords,data_channel_is_entertainment,data_channel_is_socmed,data_channel_is_tech,kw_max_min,kw_min_max,kw_max_max,kw_avg_avg,self_reference_min_shares,self_reference_avg_sharess,is_weekend,LDA_02,min_positive_polarity,title_sentiment_polarity,Label
0,219.0,5.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,496.0,496.000000,0.0,0.040005,0.100000,-0.187500,0.0
1,255.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.050096,0.033333,0.000000,0.0
2,211.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,918.0,918.000000,0.0,0.033351,0.100000,0.000000,1.0
3,531.0,7.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.494651,0.136364,0.000000,0.0
4,1072.0,7.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,545.0,3151.157895,0.0,0.028575,0.033333,0.136364,0.0


In [24]:
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession, SQLContext
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder
from pyspark.ml.tuning import CrossValidator

In [25]:
spark = (SparkSession
.builder
.appName("SparkSQLExampleApp")
.getOrCreate())

In [32]:
words_spark = ps.from_pandas(words)

In [33]:
df_spark = words_spark.to_spark()

C:\Users\vovad\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\pyspark\pandas\utils.py:975: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `to_spark`, the existing index is lost when converting to Spark DataFrame.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


In [34]:
df_spark.show(5)

+-----------------+-------------+------------------------------+-----------------------+---------------------+-----------+-----------+-----------+-----------+--------------------------+---------------------------+-----------+---------------+----------------------+-------------------------+-----+
| n_tokens_content| num_keywords| data_channel_is_entertainment| data_channel_is_socmed| data_channel_is_tech| kw_max_min| kw_min_max| kw_max_max| kw_avg_avg| self_reference_min_shares| self_reference_avg_sharess| is_weekend|         LDA_02| min_positive_polarity| title_sentiment_polarity|Label|
+-----------------+-------------+------------------------------+-----------------------+---------------------+-----------+-----------+-----------+-----------+--------------------------+---------------------------+-----------+---------------+----------------------+-------------------------+-----+
|            219.0|          5.0|                           1.0|                    0.0|                  0.0

In [35]:
trainDF, testDF = df_spark.randomSplit([.7,.3], seed=42)
print(f"""There are {trainDF.count()} rows in the training set, and {testDF.count()} in the test set""")

There are 27924 rows in the training set, and 11720 in the test set


In [36]:
trainDF.dtypes

[(' n_tokens_content', 'double'),
 (' num_keywords', 'double'),
 (' data_channel_is_entertainment', 'double'),
 (' data_channel_is_socmed', 'double'),
 (' data_channel_is_tech', 'double'),
 (' kw_max_min', 'double'),
 (' kw_min_max', 'double'),
 (' kw_max_max', 'double'),
 (' kw_avg_avg', 'double'),
 (' self_reference_min_shares', 'double'),
 (' self_reference_avg_sharess', 'double'),
 (' is_weekend', 'double'),
 (' LDA_02', 'double'),
 (' min_positive_polarity', 'double'),
 (' title_sentiment_polarity', 'double'),
 ('Label', 'double')]

In [37]:
assemblerInputs = [field for (field, dataType) in trainDF.dtypes if ((dataType == "double") & (field != "Label"))]
vecAssembler = VectorAssembler(inputCols=assemblerInputs,
                               outputCol="features") 

In [38]:
dt = DecisionTreeClassifier(labelCol="Label", featuresCol="features", seed=42)

stages = [vecAssembler, dt]

pipeline = Pipeline(stages=stages)
dt.setMaxBins(40)
pipelineModel = pipeline.fit(trainDF)

In [39]:
predDF = pipelineModel.transform(testDF)
# predDF.select("features", "Label", "prediction").show(30)

In [40]:
dtModel = pipelineModel.stages[-1]
# print(dtModel.toDebugString)

In [41]:
featureImp = pd.DataFrame(list(zip(vecAssembler.getInputCols(), dtModel.featureImportances)), columns=["feature", "importance"])
featureImp.sort_values(by="importance", ascending=False)

,feature,importance
8,kw_avg_avg,0.350485
2,data_channel_is_entertainment,0.144778
11,is_weekend,0.144438
4,data_channel_is_tech,0.102748
3,data_channel_is_socmed,0.083331
10,self_reference_avg_sharess,0.072001
12,LDA_02,0.040639
6,kw_min_max,0.016475
7,kw_max_max,0.012741
0,n_tokens_content,0.011978


In [42]:
evaluator = MulticlassClassificationEvaluator(
    labelCol="Label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predDF)
print("Test Error = %g " % (1.0 - accuracy))

Test Error = 0.358362 


In [ ]:
with mlflow.start_run(run_name="decision-tree") as run:
    # Log model
    pipelineModel = pipeline.fit(trainDF)
    mlflow.spark.log_model(pipelineModel, "Model")
    
    predDF = pipelineModel.transform(testDF)
    evaluator = MulticlassClassificationEvaluator(
    labelCol="Label", predictionCol="prediction")
    accuracy = evaluator.setMetricName('accuracy').evaluate(predDF)
    weightedPrecision = evaluator.setMetricName('weightedPrecision').evaluate(predDF)
    mlflow.log_metrics({"accuracy": accuracy, "weightedPrecision": weightedPrecision})
    dtModel = pipelineModel.stages[-1]
    featureImp = (pd.DataFrame(list(zip(vecAssembler.getInputCols(), 
                                        dtModel.featureImportances)), 
                                        columns=["feature", "importance"])
                  .sort_values(by="importance", ascending=False))
    featureImp.to_csv("feature-importance.csv", index=False)
    mlflow.log_artifact("feature-importance.csv")